In [ ]:
pip install folium

In [ ]:
import numpy as np 
import pandas as pd 
df_ecad=pd.read_csv('data/ecad_20221216.csv')
df_energy= pd.read_csv('data/energy_rating_data.csv')

In [ ]:
import folium 
from folium.plugins import Search

In [ ]:
# Joining the ecad_data with energy rating data
df_ber = pd.merge(df_ecad,df_energy, on = 'ecad_id', how = 'inner')

In [ ]:
df_ber.head()

In [ ]:
#Sligo Map
lat= 54.26969
lang= -8.46943

maps=folium.Map(location=[lat,lang],zoom_start=10)

maps

In [ ]:
#Color codes for BER rating scale
colors={
    'A1':'#00a54f',
    'A2':'#00a54f',
    'A3':'#00a54f',
    'B1':'#4cb848',
    'B2':'#4cb848',
    'B3':'#4cb848',
    'C1':'#bed630',
    'C2':'#bed630',
    'C3':'#bed630',
    'D1':'#fff101',
    'D2':'#fff101',
    'E1':'#fcb814',
    'E2':'#fcb814',
    'F':'#f36e21',
    'G':'#ee1d23',
}

In [ ]:
# Creating the popup labels for energy rating data for building id to display on selection of the marker in the map.

def popup_html(row):
    i = row
    Building_ID=df_ber['building_id'].iloc[i] 
    Address= df_ber['address_line_1'].iloc[i]
    Ber_Rating = df_ber['ber_rating'].iloc[i]
    Ber_Rating_Kwh = df_ber['ber_rating_kwh'].iloc[i]
    Co2_Emission = df_ber['co2_emission'].iloc[i]

    left_col_color = "#808080"
    right_col_color = "#dcdcdc"
    
    html = """
<!DOCTYPE html>
<html>
<center><h4 style="margin-bottom:5"; width="200px">{}</h4>""".format(Building_ID) + """</center>
<center> <table style="height: 126px; width: 305px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Address </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">"""+ Address + """</td>
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ber_Rating </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Ber_Rating) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Ber_Rating_Kwh </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Ber_Rating_Kwh) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Co2_Emission </span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(Co2_Emission) + """
</tr>
</tbody>
</table></center>
</html>
"""
    return html

In [ ]:
#Creating lagends

def add_legend(maps, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_label = dict(zip(labels, colors))
    
    legends = ""     
    for label, color in color_label.items():
        legends += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='legend' class='legend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legends}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var Layout = (function() {{
                    var flag = false;
                    return function() {{
                        if (!flag) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!flag)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          flag = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        Layout()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .legend {
        z-index:9999;
        float:right;
        background-color: rgb(255, 255, 255);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .legend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .legend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .legend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .legend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .legend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .legend a {
        color: #777;
        }
    </style>
    """

    maps.get_root().header.add_child(folium.Element(script + css))

    return maps

In [ ]:
#Plotting BER_rating data

sligo_circle=folium.Map(location=[lat,lang],zoom_start=10)

for d in df_ber.iterrows():
        html = popup_html(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.CircleMarker(
                    [d[1]["etrs89_lat"], d[1]["etrs89_long"]],
                    radius=6,
                    color=colors[d[1]["ber_rating"]],
                    fill=True,
                    fill_color=colors[d[1]["ber_rating"]],
                    fill_opacity=0.7,
                    popup=popup
            ).add_to(sligo_circle)

sligo_circle = add_legend(sligo_circle, 'Building Energy Rating', colors = list(colors.values()), labels = list(colors.keys()))
sligo_circle.save('Sligo_Energy_Rating_CircleMarker.html')
sligo_circle


In [ ]:
#Search cluster
Sligo_cluster=folium.Map(location=[lat,lang],zoom_start=10)
from folium import plugins
cluster=plugins.MarkerCluster().add_to(Sligo_cluster)

for d in df_ber.iterrows(): 
        html = popup_html(d[0])
        popup = folium.Popup(folium.Html(html, script=True), max_width=500)
        folium.Marker(location=[d[1]["etrs89_lat"], d[1]["etrs89_long"]], popup=popup,name=d[1]["building_id"]).add_to(cluster)
Search(cluster,search_label='name',placeholder='Search for building ID').add_to(Sligo_cluster)            
Sligo_cluster.save('Sligo_ClusterView.html')
Sligo_cluster
